In [1]:
import geemap
import datetime
import ee
import time
import os
import sys


Please restart Jupyter kernel after installation if you encounter any errors when importing geemap.


ModuleNotFoundError: No module named 'fcntl'

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project="259532508364")  # Replace with your actual project ID


In [ ]:
# Coordonnées approximatives pour couvrir une large zone du RDC
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
congo = countries.filter(ee.Filter.eq('country_na', 'Democratic Republic of the Congo'))
# Définir une zone approximative en RDC avec un polygone personnalisé
roi = ee.Geometry.Polygon([
    [23, -4],
    [30, -4],
    [30, 3],
    [23, 3],

])


In [ ]:
# Define the time range for data acquisition
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2015, 12, 31)

# Load Landsat 8 data with the correct collection ID
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))


In [ ]:
# Select the bands for NDVI calculation
ndvi = collection.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))

# Calculate the mean NDVI over the time period
mean_ndvi = ndvi.mean()

# Define visualization parameters for NDVI
vis_params = {
    'min': -0.2,  # Ajuster la valeur minimale
    'max': 0.8,   # Ajuster la valeur maximale
    'palette': ['blue', 'white', 'green']
}


In [ ]:
# Calculate bounds from ROI
bounds = roi.bounds().getInfo()['coordinates'][0]
xmin = min(coord[0] for coord in bounds)
xmax = max(coord[0] for coord in bounds)
ymin = min(coord[1] for coord in bounds)
ymax = max(coord[1] for coord in bounds)
width = xmax - xmin
height = ymax - ymin

#Création de la carte
Map = geemap.Map(center=[ymin + height/2, xmin + width/2], zoom=9)

# Ajout des données Landsat NDVI à la carte
Map.addLayer(mean_ndvi, vis_params, 'Mean NDVI')

# Affichage de la carte
Map.addLayer(roi, {'color': 'red'}, 'ROI')  # Ajout de la ROI à la carte
Map.centerObject(roi, 10)
Map.addLayerControl()  # Ajout du widget de contrôle des couches
Map

Map(center=[-14.329829933489219, -54.00000000000001], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Define the time range for data acquisition
start_date = datetime.datetime(2021, 1, 1)
end_date = datetime.datetime(2021, 12, 31)

# Load Landsat 8 data with the correct collection ID
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

# Select the bands for NDVI calculation
ndvi = collection.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))

# Calculate the mean NDVI over the time period
mean_ndvi = ndvi.mean()

# Define visualization parameters for NDVI
vis_params = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['blue', 'white', 'green']
}


In [ ]:
# Create a map
Map = geemap.Map(center=[ymin + height/2, xmin + width/2], zoom=1)

# Add Landsat NDVI data to the map
Map.addLayer(mean_ndvi, vis_params, 'Mean NDVI')

# Display the map
Map.addLayer(roi, {'color': 'red'}, 'ROI')  # Add ROI to the map
Map.centerObject(roi, 10)
Map.addLayerControl()  # Add layer control widget
Map


Map(center=[-14.329829933489219, -54.00000000000001], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
#Data colecting date
start_date_2 = '2015-01-01'
end_date_2 = '2025-01-31'

# Time interval
interval_day = 270

# Function to generate date list at regular interval
def generate_date_range(start_date, end_date, interval_day):
  """Generate a list of dates between two dates given, with a specifique interval"""
  start = datetime.datetime.strptime(start_date, '%Y-%m-%d')
  end = datetime.datetime.strptime(end_date, '%Y-%m-%d')
  date_list = []
  while start <= end:
    date_list.append(start.strftime('%Y-%m-%d'))
    start += datetime.timedelta(days=interval_day)
  return date_list

# Generate the date list
date_list = generate_date_range(start_date_2, end_date_2, interval_day)
print(date_list)

['2015-01-01', '2015-09-28', '2016-06-24', '2017-03-21', '2017-12-16', '2018-09-12', '2019-06-09', '2020-03-05', '2020-11-30', '2021-08-27', '2022-05-24', '2023-02-18', '2023-11-15', '2024-08-11']


In [ ]:
# Show images one by one for each date
from IPython.display import clear_output, display
import time

# Set display delay in seconds
DISPLAY_DELAY = 1  # Adjust this value to change how long each map is displayed

for date in range(len(date_list)):
    start_date = date_list[date]
    if date + 1 < len(date_list):
        end_date = date_list[date + 1]
    else:
        break
    print(f"Processing date range: {start_date} to {end_date}")

    collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUD_COVER', 20))

    # Check if the collection is empty
    if collection.size().getInfo() == 0:
        print(f"No images found for date range: {start_date} to {end_date}")
        continue

    # Select the bands for NDVI calculation
    ndvi = collection.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))

    # Calculate the mean NDVI over the time period
    mean_ndvi = ndvi.mean()

    # Define visualization parameters for NDVI
    vis_params = {
        'min': -0.2,
        'max': 0.8,
        'palette': ['blue', 'white', 'green']
    }

    # Create a map
    Map = geemap.Map(center=[ymin + height/2, xmin + width/2], zoom=1)

    # Add Landsat NDVI data to the map
    Map.addLayer(mean_ndvi, vis_params, 'Mean NDVI')

    # Display the map
    Map.addLayer(roi, {'color': 'red'}, 'ROI')  # Add ROI to the map
    Map.centerObject(roi, 10)
    Map.addLayerControl()  # Add layer control widget

    # Clear the output to show the new map
    clear_output(wait=True)
    display(Map)  # Display the map
    print(f"Displaying map for date range: {start_date} to {end_date}")
    print(f"Next map will be shown in {DISPLAY_DELAY} seconds...")
    time.sleep(DISPLAY_DELAY)  # Wait for specified seconds before showing next map
    
    # Grid size 1250m
    grid_size = 1250
    
    # Divide the zone of interest (ROI) into sub-zones
    sub_zones = roi.coveringGrid("EPSG:4326", grid_size)

    # Calculate the mean NDVI for each sub-zone
    ndvi_results = mean_ndvi.reduceRegions(
        collection=sub_zones,
        reducer=ee.Reducer.mean(),
        scale=grid_size,
        # Limite the number of pixels treated to avoid memory issues
    )

    # Export the results to Google Drive in CSV format
    task = ee.batch.Export.table.toDrive(
        collection=ndvi_results,
        description=f'NDVI_SubZones_Export_{start_date}_{end_date}',
        folder='NDVI_Results_RDC',  # Specify the folder name in your Google Drive
        fileFormat='CSV'
    )

    task.start()  # Start the export task
    print(f"Exporting NDVI results to Google Drive for date range: {start_date} to {end_date}...")
    # Wait for the task to complete
    while task.active():
        print('Exporting...\nPolling for task (id: {}).'.format(task.id))
        time.sleep(2.5)  # Wait for 10 seconds before checking again
    task_status = task.status()
    print('Export task status: {}'.format(task_status['state']))
    if task_status['state'] == 'COMPLETED':
        print(f"Export completed successfully for date range: {start_date} to {end_date}.")
    else:
        print(f"Export failed for date range: {start_date} to {end_date}.")
        
    

Map(center=[-14.329829933489219, -54.00000000000001], controls=(WidgetControl(options=['position', 'transparen…

Exporting NDVI results to Google Drive for date range: 2023-11-15 to 2024-08-11...
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Exporting...
Polling for task (id: ZZK56BS6SGQNBEDN7Z7EH4JQ).
Export task status: COMPLETED
Export completed successfully for date range: 2023-11-15 to 2024-08-11.


TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

...

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.